In [38]:
##Problem Set 5

In [141]:
## import the packages
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import differential_evolution
from geopy.distance import geodesic

In [142]:
# Read in the datafile
# 
df = pd.read_csv(r'C:\Users\PC\Desktop\radio_merger_data.csv')

In [146]:
df.head(n = 10)

,year,buyer_id,target_id,buyer_lat,buyer_long,target_lat,target_long,price,hhi_target,num_stations_buyer,population_target,corp_owner_buyer,price_per_mil,population_per_mil
0,2007,1,1,46.592512,-92.549564,44.375073,-92.039543,157763.91,80,3,21676,0,0.157764,0.021676
1,2007,2,2,32.578185,-85.349003,33.025375,-86.059702,1472463.20,376,1,11539,0,1.472463,0.011539
2,2007,3,3,30.639867,-88.254450,31.122499,-87.766408,3786333.90,129,1,182265,0,3.786334,0.182265
3,2007,4,4,38.956806,-94.683236,36.196946,-94.006823,473291.74,188,20,203065,0,0.473292,0.203065
4,2007,5,5,41.054082,-73.536216,40.909898,-73.457023,1840579.00,284,0,1493350,0,1.840579,1.493350
5,2007,6,6,39.223416,-106.882770,39.269849,-107.023350,1104347.40,51,1,17148,0,1.104347,0.017148
6,2007,7,7,39.863544,-96.617219,39.442281,-98.050660,184057.90,58,4,6373,0,0.184058,0.006373
7,2007,8,8,38.819795,-121.277440,41.972979,-87.947120,17879910.00,82,208,916924,0,17.879910,0.916924
8,2007,9,9,33.786594,-118.298660,28.709332,-80.856849,25242226.00,34,37,543376,0,25.242226,0.543376
9,2007,10,10,42.443224,-114.629500,44.533039,-115.858440,946583.47,50,4,9862,0,0.946583,0.009862


In [147]:
#Create datasets for year 2007 and 2008
df2007=df[df['year'] == 2007 ]
df2008=df[df['year'] == 2008 ]

In [150]:
# Calculate distances for years and the result in a matrix
def calc_distance(distance,buyer_lat, buyer_long, target_lat, target_long, start_index):
    for i in range(distance.shape[0]):
        for j in range(distance.shape[0]):
            distance[i,j] = geodesic((buyer_lat[i + start_index], buyer_long[i + start_index]),
                                    (target_lat[j + start_index],target_long[j + start_index])).miles
    return distance

# seperate buyers and targets into different groups by year and select their location
buyer_lat2007 = df2007.loc[:, 'buyer_lat']
buyer_long2007 = df2007.loc[:, 'buyer_long']
target_lat2007 = df2007.loc[:, 'target_lat']
target_long2007 = df2007.loc[:, 'target_long']

buyer_lat2008 = df2008.loc[:, 'buyer_lat']
buyer_long2008 = df2008.loc[:, 'buyer_long']
target_lat2008 = df2008.loc[:, 'target_lat']
target_long2008 = df2008.loc[:, 'target_long']

#calculate the distance
distance2007 = np.zeros((df2007.shape[0],df2007.shape[0]))
distance2007 = calc_distance(distance2007, buyer_lat2007,buyer_long2007, target_lat2007,target_long2007, 0)
distance2008 = np.zeros((df2008.shape[0],df2008.shape[0]))
distance2008 = calc_distance(distance2008, buyer_lat2008,buyer_long2008, target_lat2008,target_long2008, 45)

In [163]:
# Define the MSE function
def mse(param, buyer_nsb, buyer_cob, population_per_mil, distance, start_index):
    '''
    param[0]:alpha, param[1]:beta
    '''
    result = 0
    estimate_fomula = np.zeros((distance.shape[0],distance.shape[0]))
    for i in range(distance.shape[0]):
        for j in range(distance.shape[0]):
            estimate_fomula[i,j] = buyer_nsb[i + start_index] * population_per_mil[j + start_index] + param[0] * buyer_cob[i + start_index] * population_per_mil[j + start_index] + param[1] * distance[i, j]
    
    for i in range(distance.shape[0] - 1):
        for j in range(i + 1, distance.shape[0]):
            if ((estimate_fomula[i,i] + estimate_fomula[j,j]) > (estimate_fomula[i,j] + estimate_fomula[j,i])):
                result = result + 1
    return -result

buyer_nsb_2007 = df2007.loc[:, 'num_stations_buyer']
buyer_cob_2007 = df2007.loc[:, 'corp_owner_buyer']
population_per_mil_2007 = df2007.loc[:,'population_target'] / 1000000

bounds = [(-1000,1000),(-1000,1000)]
arguments2007 = (buyer_nsb_2007, buyer_cob_2007, population_per_mil_2007, distance2007, 0)
max_result2007 = differential_evolution(mse, bounds, args = arguments2007, strategy = 'best1bin', maxiter = 10000)

buyer_nsb_2008 = df2008.loc[:, 'num_stations_buyer']
buyer_cob_2008 = df2008.loc[:, 'corp_owner_buyer']
population_per_mil_2008 = df2008.loc[:,'population_target'] / 1000000

bounds = [(-1000,1000),(-1000,1000)]
arguments2008 = (buyer_nsb_2008, buyer_cob_2008, population_per_mil_2008, distance2008, 45)
max_result2008 = differential_evolution(mse, bounds, args = arguments2008, strategy = 'best1bin', maxiter = 10000)

#print the result
print(max_result2007.x, max_result2008.x)

[402.69862708 771.65597602] [140.83911066 637.65178431]


In [185]:
#Define the second MSE function
def mse_2(param, buyer_nsb, buyer_cob, hhi_target ,population_per_mil, price_per_mil, distance, start_index):
    '''
    param[0]:delta, param[1]:alpha, param[2]:gamma, param[3]:beta
    '''
    result = 0
    estimate_fomula = np.zeros((distance.shape[0],distance.shape[0]))
    for i in range(distance.shape[0]):
        for j in range(distance.shape[0]):
            estimate_fomula[i,j] = param[0] * buyer_nsb[i + start_index] * population_per_mil[j + start_index] + param[1] * buyer_cob[i + start_index] * population_per_mil[j + start_index] + param[2] * hhi_target[j + start_index] + param[3] * distance[i, j]
    
    for i in range(distance.shape[0] - 1):
        for j in range(i + 1, distance.shape[0]):
            if ((estimate_fomula[i,i] - estimate_fomula[i,j]) > (price_per_mil[i + start_index] - price_per_mil[j + start_index]) and (estimate_fomula[j,j] - estimate_fomula[j,i]) > price_per_mil[j + start_index] - price_per_mil[i + start_index] ):
                result = result + 1
    return -result

## for the year 2007
price_per_mil_2007 = df2007.loc[:, 'price'] / 1000000
hhi_target2007 = df2007.loc[:, 'hhi_target']

bounds = [(-1000,1000),(-1000,1000),(-1000,1000),(-1000,1000)]
arguments2007 = (buyer_nsb_2007, buyer_cob_2007, hhi_target2007, population_per_mil_2007, price_per_mil_2007, distance2007, 0)
max_result2007 = differential_evolution(mse_2, bounds, args = arguments2007, strategy = 'best1bin', maxiter = 10000)

## for the year 2008
price_per_mil_2008 = df2008.loc[:,'price'] / 1000000
hhi_target2008 = df2008.loc[:, 'hhi_target']

bounds = [(-1000,1000),(-1000,1000),(-1000,1000),(-1000,1000)]
arguments2008 = (buyer_nsb_2008, buyer_cob_2008, hhi_target2008, population_per_mil_2008, price_per_mil_2008, distance2008, 45)
max_result2008 = differential_evolution(mse_2, bounds, args = arguments2008, strategy = 'best1bin', maxiter = 10000)

#print the result
print(max_result2007.x, max_result2008.x)

[ 6.06814571e+02 -4.43770279e+02 -1.33165436e-01 -6.01644447e+01] [-892.65055141  625.73495182   46.55977302 -253.11990843]
